# Libraries

In [1]:
import numpy as np
import scipy.sparse as ss
import pickle
import pandas as pd
import os 

In [2]:
from TELF.pre_processing import Vulture
from TELF.pre_processing.Vulture.modules import SimpleCleaner
from TELF.pre_processing.Vulture.modules import LemmatizeCleaner
from TELF.pre_processing.Vulture.modules import RemoveNonEnglishCleaner
from TELF.pre_processing.Vulture.default_stop_words import STOP_WORDS
from TELF.pre_processing.Vulture.default_stop_phrases import STOP_PHRASES

In [3]:
from TELF.factorization.HNMFk import HNMFk

In [4]:
from TELF.pre_processing import Beaver

In [5]:
from TELF.post_processing import ArcticFox

/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [6]:
df = pd.read_csv(os.path.join("..", "..", "data", "sample2.csv"))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   eid               235 non-null    object 
 1   s2id              230 non-null    object 
 2   doi               235 non-null    object 
 3   title             235 non-null    object 
 4   abstract          232 non-null    object 
 5   year              235 non-null    int64  
 6   authors           235 non-null    object 
 7   author_ids        235 non-null    object 
 8   affiliations      235 non-null    object 
 9   funding           109 non-null    object 
 10  PACs              95 non-null     object 
 11  publication_name  235 non-null    object 
 12  subject_areas     235 non-null    object 
 13  s2_authors        230 non-null    object 
 14  s2_author_ids     230 non-null    object 
 15  citations         201 non-null    object 
 16  references        191 non-null    object 
 1

# Clean Text

In [7]:
steps = [
    RemoveNonEnglishCleaner(ascii_ratio=0.9, stopwords_ratio=0.25),
    SimpleCleaner(stop_words = STOP_WORDS,
                  stop_phrases = STOP_PHRASES,
                  order = [
                      'standardize_hyphens',
                      'isolate_frozen',
                      'remove_copyright_statement',
                      'remove_stop_phrases',
                      'make_lower_case',
                      'remove_formulas',
                      'normalize',
                      'remove_next_line',
                      'remove_email',
                      'remove_()',
                      'remove_[]',
                      'remove_special_characters',
                      'remove_nonASCII_boundary',
                      'remove_nonASCII',
                      'remove_tags',
                      'remove_stop_words',
                      'remove_standalone_numbers',
                      'remove_extra_whitespace',
                      'min_characters',
                  ]
                 ),
    LemmatizeCleaner('spacy'),
]

In [8]:
vulture = Vulture(n_jobs=1, verbose=10)
df = vulture.clean_dataframe(df=df, 
                        columns=["abstract", "title"],
                        append_to_original_df=True,
                        concat_cleaned_cols=True
                        )

[Vulture]: Cleaning 235 documents
100%|██████████| 1/1 [00:00<00:00,  1.65it/s]
/Users/maksim/Desktop/Code/telf_internal/TELF/pre_processing/Vulture/vulture.py:264: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace('', np.nan, inplace=True)  # set null entries to None


In [9]:
df.clean_abstract_title

0      vulnerabilities pose threat cybersecurity kern...
1      cybersecurity frameworks nist guidelines risk ...
2      bias-variance tradeoff determines models abili...
3      efficient distributed implementation truncated...
4      autoencoders dimensionality reduction anomaly ...
                             ...                        
230    transfer learning models generalize leveraging...
231    explainable ai xai enhances trust ai models ma...
232    differential privacy ensures machine learning ...
233    recurrent neural networks sequential speech te...
234    transfer learning pre-trained models fine-tune...
Name: clean_abstract_title, Length: 235, dtype: object

# Build The Vocabulary and the Document-Term Matrix

In [10]:
DATA_COLUMN = 'clean_abstract_title'
RESULTS = "result_example"
HIGHLIGHT_WORDS = ['analysis', 'tensor']
HIGHLIGHT_WEIGHTS = [2 for i in HIGHLIGHT_WORDS]
beaver = Beaver()
os.makedirs(RESULTS, exist_ok=True)
settings = {
    "dataset" : df,
    "target_column" : DATA_COLUMN,
    'highlighting': HIGHLIGHT_WORDS,
    'weights':HIGHLIGHT_WEIGHTS,
    "matrix_type" : "tfidf",
    "save_path" : RESULTS
}
X, vocabulary = beaver.documents_words(**settings)

In [11]:
X = X.T.tocsr()
X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 25047 stored elements and shape (696, 235)>

In [12]:
assert X.shape[1] == len(df)

In [13]:
vocabulary[:10]

array(['128pb', '1e', '1tb', '32x', '76x', 'ability', 'abstaining',
       'accelerated', 'accuracy', 'accurate'], dtype=object)

In [14]:
len(vocabulary)

696

# Factorize with HNMFk

In [15]:
# Define the range of cluster numbers (K) to search over
Ks = np.arange(2, 30, 1)  # From 2 to 29 inclusive

# Number of perturbations and iterations to run
perts = 2  # Number of perturbed runs to estimate stability
iters = 2  # Number of iterations for each perturbation

# Small perturbation epsilon added to input data
eps = 0.025

# Initialization method for NMF (Non-negative Matrix Factorization)
init = "nnsvd"  # Nonnegative SVD initialization

# Path to save HNMFk results
HNMFK_save_path = os.path.join(RESULTS, "example_HNMFK")
name = HNMFK_save_path  # Alias for convenience

# Parameters for HNMFk (Hierarchical Nonnegative Matrix Factorization k-search)
nmfk_params = {
    "k_search_method": "bst_pre",             # Method for determining optimal k (e.g., binary search with pre-checks)
    "sill_thresh": 0.7,                       # Silhouette threshold to accept a given k
    "H_sill_thresh": 0.05,                    # Threshold for H-matrix silhouette to refine k selection
    "n_perturbs": perts,                      # Number of perturbations
    "n_iters": iters,                         # Number of iterations per perturbation
    "epsilon": eps,                           # Perturbation strength
    "n_jobs": -1,                             # Use all available CPU cores
    "init": init,                             # NMF initialization method
    "use_gpu": False,                         # Whether to use GPU acceleration
    "save_path": HNMFK_save_path,             # Directory where results will be saved
    "predict_k_method": "WH_sill",            # Method to predict k using W and H matrix silhouettes
    "predict_k": True,                        # Whether to automatically predict k
    "verbose": False,                          # Verbose output
    "nmf_verbose": False,                     # Verbose output from NMF algorithm
    "transpose": False,                       # Whether to transpose input data
    "pruned": True,                           # Whether to prune unstable clusters
    "nmf_method": "nmf_fro_mu",               # NMF solver method (Frobenius norm, multiplicative updates)
    "calculate_error": False,                 # Whether to calculate reconstruction error
    "use_consensus_stopping": 0,              # Whether to use consensus stopping (0 = off)
    "calculate_pac": False,                   # Whether to compute PAC (proportion of ambiguous clustering)
    "consensus_mat": False,                   # Whether to generate consensus matrix
    "perturb_type": "uniform",                # Type of perturbation (e.g., uniform noise)
    "perturb_multiprocessing": False,         # Use multiprocessing during perturbation
    "perturb_verbose": False,                 # Verbose output during perturbation
    "simple_plot": True                       # Whether to generate simplified summary plots
}


In [16]:
class CustomSemanticCallback:
    def __init__(self, 
                 df: pd.DataFrame, 
                 target_column=DATA_COLUMN,
                 options={'vocabulary': vocabulary},
                 matrix_type="tfidf") -> None:
        """
        Initializes the callback with a DataFrame and matrix generation settings.

        Parameters:
        - df: The full DataFrame containing the text data.
        - target_column: Column name containing the target text to vectorize (default is a global DATA_COLUMN).
        - options: Options dictionary passed to Beaver (e.g., fixed vocabulary, token settings).
        - matrix_type: Type of vectorization matrix (e.g., "tfidf", "count").
        """
        self.df = df
        self.target_column = target_column
        self.options = options
        self.matrix_type = matrix_type

    def __call__(self, original_indices: np.ndarray):
        """
        Callable interface for dynamically generating document-term matrices 
        from a subset of the DataFrame.

        Parameters:
        - original_indices: Numpy array of row indices from self.df to subset and transform.

        Returns:
        - Tuple of (X, metadata), where:
            - X is a document-term sparse matrix (CSR format).
            - metadata is a dict containing either 'vocab' or a 'stop_reason' if failed.
        """
        current_beaver = Beaver()  # Initialize a new instance of the Beaver text vectorizer

        # Extract the subset of the DataFrame using the provided indices
        current_df = self.df.iloc[original_indices].copy()

        # Construct parameters for the Beaver vectorizer
        current_beaver_matrix_settings = {
            "dataset": current_df,
            "target_column": self.target_column,
            "options": self.options,
            "highlighting": HIGHLIGHT_WORDS,     # Global list of words to highlight
            "weights": HIGHLIGHT_WEIGHTS,        # Associated weights for highlighting
            "matrix_type": self.matrix_type,     # Type of matrix to construct (e.g., TF-IDF)
            "save_path": None                    # No file output; matrix is returned
        }

        try:
            # Attempt to generate the document-word matrix
            current_X, vocab = current_beaver.documents_words(**current_beaver_matrix_settings)
            
            # Transpose to get documents as rows (CSR format is efficient for row slicing)
            current_X = current_X.T.tocsr()
            
            return current_X, {'vocab': vocab}

        except:
            # On failure, return a 1x1 matrix to signal a stopping condition for downstream tasks
            csr_matrix = ss.csr_matrix([[1]])
            return csr_matrix, {'stop_reason': "documents_words couldn't make matrix"}

In [17]:
# Parameters for initializing and training the HNMFk model
hnmfk_params = {
    "n_nodes": 1,  # Number of root nodes to begin with (can grow as depth increases)
    
    # List of NMF parameters for the top-level (depth=0); can use different sets for different nodes
    "nmfk_params": [nmfk_params],  
    
    # Callable that generates a document-term matrix from a subset of the DataFrame (dynamic input for each node)
    "generate_X_callback": CustomSemanticCallback(df=df, options={'vocabulary': vocabulary}),
    
    "cluster_on": "H",  # Which factor matrix to use for clustering (H = document-topic)
    
    "depth": 2,  # Depth of the hierarchy; e.g., 2 means root + one layer of children
    
    "sample_thresh": 3,  # Minimum number of samples required to split/cluster a node further
    
    "K2": False,  # If True, forces all subclusters to use k=2; here we allow varying k
    
    # Range of K to try for deeper layers (children nodes)
    "Ks_deep_min": 1,
    "Ks_deep_max": 30,
    "Ks_deep_step": 1,
    
    "experiment_name": name,  # Folder/identifier for saving results and checkpoints
}

# Instantiate the HNMFk model with the above parameters
model = HNMFk(**hnmfk_params)

# Fit the model on matrix X using the specified range of Ks
# - from_checkpoint: load previously saved progress if available
# - save_checkpoint: periodically save progress for recovery or inspection
model.fit(X, Ks, from_checkpoint=False, save_checkpoint=True)

# Traverse and collect all nodes created in the hierarchical model
all_nodes = model.traverse_nodes()
print(len(all_nodes))  # Output the total number of nodes (clusters at all levels)

# Save the full trained model to a pickle file for reuse or inspection
with open(os.path.join('result_example', 'HNMFK_highlight.pkl'), 'wb') as output_file:
    pickle.dump(model, output_file)


/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Done
103


# Post-process with Arctic Fox

In [18]:
# Load a pre-trained HNMFk model from disk
model = HNMFk(experiment_name=os.path.join("result_example", "example_HNMFK"))
model.load_model()  # Loads model from the provided experiment_name path

# Initialize ArcticFox pipeline
# - model: the hierarchical clustering model (HNMFk)
# - embedding_model: name of the sentence embedding model used for label generation
# - clean_cols_name: column in the DataFrame containing the cleaned text input
pipeline = ArcticFox(
    model=model,
    embedding_model="SCINCL",        # Example: SCINCL embedding model fine-tuned for scientific text
    clean_cols_name=DATA_COLUMN      # The text column used for label generation and analysis
)

# Run the full ArcticFox pipeline:
# This handles hierarchical cluster labeling, statistics generation, data collection, and label propagation
pipeline.run_full_pipeline(
    vocab=vocabulary,                # Vocabulary used to guide or filter cluster content
    data_df=df,                      # Original dataset (same used in HNMFk)
    ollama_model="llama3.2:3b-instruct-fp16",  # Language model used for semantic label generation
    label_clusters=True,             # Enable automatic labeling of clusters
    generate_stats=True,             # Generate cluster-level statistics
    collect_data=True,               # Gather node-wise structured data (e.g., keywords, size)
    process_parents=True,            # Propagate labels or stats upward through the hierarchy
    skip_completed=True,             # Skip processing of nodes already labeled/stored
    label_criteria={                 # Rules to filter generated labels
        "minimum words": 2,
        "maximum words": 6
    },
    label_info={                     # Additional metadata to associate with generated labels
        "source": "Science"
    },
    number_of_labels=5               # Number of candidate labels to generate per node
)


Loading saved object state from checkpoint...
Step 1: Post-processing W/H matrix and cluster data...


100%|██████████| 2/2 [00:00<00:00, 27869.13it/s]


100%|██████████| 3/3 [00:00<00:00, 32513.98it/s]


100%|██████████| 2/2 [00:00<00:00, 32768.00it/s]


100%|██████████| 1/1 [00:00<00:00, 17848.10it/s]


...

100%|██████████| 7/7 [00:00<00:00, 56899.47it/s]


100%|██████████| 12/12 [00:00<00:00, 108240.10it/s]


100%|██████████| 25/25 [00:00<00:00, 109569.07it/s]


100%|██████████| 52/52 [00:00<00:00, 104007.54it/s]


100%|██████████| 25/25 [00:00<00:00, 11877.84it/s]
